In [38]:
!pip install stop_words -q


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
# !python -m spacy download en_core_web_sm -q

In [62]:
import pandas as pd
import re
import spacy
import subprocess
from stop_words import get_stop_words
from nltk.corpus import stopwords

2023-05-15 23:13:28.436808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [47]:
df = pd.read_csv('../data/assembled.csv', index_col=0)

### Prepare the text for word2vec (and for any other future technique)

In [41]:
stop_words = list(get_stop_words('en'))     
nltk_words = list(stopwords.words('english')) 

stop_words = list(set(stop_words + nltk_words))

In [42]:
def preprocess(text):
    text = re.sub(r'[^\w\s\'.\-\+]', ' ', text)  # I prefer to leave some signs (`.`, `+`, `-`) as the astro objs may contain those
    text = re.sub(r'[ \n]+', ' ', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text.strip().lower()

In [49]:
df['body_clean'] = df.body.apply(preprocess)

In [61]:
df.body_clean[5]  # looks better

'on december 28 1997 kundic lubin obtained spectra optical transient grb 971214 iauc 6788 low resolution imaging spectrograph lris mounted keck ii telescope. the seeing conditions excellent. if transient continued power-law decay indicated data halpern et al. iauc 6788 epoch light position dominated host cf. kulkarni et al. gcn 27 atel 5 . analysis spectra show slightly extended emission feature 5384 a. additionally broad absorption feature seen 5752 a. we time make definitive statement redshift host. if emission feature identified lyman-alpha absorption feature oi 1302 a redshift presumed host z 3.43. however appears obvious depression bluewards feature normally seen high-redshift objects caused lyman forest . if however emission feature identified oii 3727 line redshift z 0.44. a strong emission feature wavelength 5602 a also seen spectrum nearby galaxy g1 gcn 12 gcn 13 . identifying oii 3727 a feature well matching major absorption features yields redshift z 0.50. we process analyzi

In [54]:
df['body_clean'].to_csv("clean_bodies.csv", index=True)

### Save to txt to train Floret embeddings
Floret is a faster implementation of a FastText

In [55]:
with open('floret/train_data.txt', 'w') as f:
    for txt in df['body_clean'].values:
        f.write(f'{txt}\n')

### Call the training script in floret dir

In [57]:
# subprocess.run("python train_floret.py train_data.txt vectors")

come back in ~20 minutes and check for the vectors
#### made 2 versions of the vectors - with dim 256 for the NER training, and with dim 128 to later embed the extracted entities 

In [99]:
!python -m spacy init vectors en floret/emb_256/vectors.vec floret-256

ℹ Creating blank nlp object for language 'en'
[2023-05-16 16:45:39,121] [INFO] Reading vectors from floret/emb_256/vectors.vec
342028it [00:39, 8715.64it/s] 
[2023-05-16 16:46:18,405] [INFO] Loaded vectors from floret/emb_256/vectors.vec
✔ Successfully converted 342028 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/Users/vladkutsuruk/Documents/Diploma/word2vec/floret-256


In [100]:
floret_256 = spacy.load("floret-256")

In [101]:
floret_256.add_pipe("ner", source=spacy.load("en_core_web_sm"))
floret_256.add_pipe("tok2vec", source=spacy.load("en_core_web_sm"))

/Users/vladkutsuruk/opt/anaconda3/lib/python3.9/site-packages/spacy/language.py:705: UserWarning: [W113] Sourced component 'tok2vec' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=source_name))


In [102]:
floret_256.to_disk("floret-256")  # add pipes from existing model 

In [103]:
floret_256("Cos-B").vector.shape

(256,)

### and the 128-d vectors model

In [69]:
!python -m spacy init vectors en floret/emb_128/vectors.vec floret-128

ℹ Creating blank nlp object for language 'en'
[2023-05-15 23:33:57,882] [INFO] Reading vectors from floret/emb_128/vectors.vec
342028it [00:08, 40364.98it/s]
[2023-05-15 23:34:06,375] [INFO] Loaded vectors from floret/emb_128/vectors.vec
✔ Successfully converted 342028 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/Users/vladkutsuruk/Documents/Diploma/word2vec/floret-128


In [70]:
floret_128 = spacy.load("floret-128")
floret_128("Romulus").vector.shape

(128,)